This is problem set #4, in which we want you to integrate your knowledge of data wrangling with some basic simulation skills. It's a short problem set to help you get your feet wet in testing statistical concepts through "making up data" rather than consulting a textbook or doing math. 

For ease of reading, please separate your answers from our text by marking our text with the `>` character (indicating quotes). 


In [1]:
import pandas as pd
import numpy as np
import plotnine as gg
from scipy import stats
import seaborn as sns

# Part 1: ggplot practice

This part is a warmup, it should be relatively straightforward `ggplot2` practice.

Load data from Frank, Vul, Saxe (2011, Infancy), a study in which we measured infants' looking to hands in moving scenes. There were infants from 3 months all the way to about two years, and there were two movie conditions (`Faces_Medium`, in which kids played on a white background, and `Faces_Plus`, in which the backgrounds were more complex and the people in the videos were both kids and adults). An eye-tracker measured children's attention to faces. This version of the dataset only gives two conditions and only shows the amount of looking at hands (other variables were measured as well). 

```{r}
fvs <- read_csv("data/FVS2011-hands.csv")
```


In [2]:
fvs = pd.read_csv("data/FVS2011-hands.csv")

First, use `ggplot` to plot a histogram of the ages of children in the study. NOTE: this is a repeated measures design, so you can't just take a histogram of every measurement. 

```{r}
```

In [3]:
sns.distplot(fvs.age)

NameError: name 'sns' is not defined


Second, make a scatter plot showing hand looking as a function of age and condition. Add appropriate smoothing lines. Take the time to fix the axis labels and make the plot look nice.


In [ ]:
g = sns.lmplot(y='hand.look', x='age', hue='condition', data=fvs)
g = g.set_axis_labels("Age", "Hand Looking Amount")
for t, l in zip(g._legend.texts, ["Faces Medium", "Faces Plus"]): t.set_text(l)

What do you conclude from this pattern of data?

> Children's hand looking time positively correlates to their age. For movies with more complex scenes, the correlation seems to be stronger.


What statistical analyses would you perform here to quantify these differences?



>  I would compute a linear regression t-test on the coorelation between hand looking amount and age under both conditions. The null-hypothesis would be that the slopes in both conditions are the same.


In [ ]:
# Part 2: Simulation


Let's start by convincing ourselves that t-tests have the appropriate false positive rate. Run 10,000 t-tests with standard, normally-distributed data from a made up 30-person, single-measurement experiment (the command for sampling from a normal distribution is `rnorm`).

The goal of these t-tests are to determine, based on 30 observations, whether the underlying distribution (in this case a normal distribution with mean 0 and standard deviation 1) has a mean that is different from 0. In reality, the mean is not different from 0 (we sampled it using `rnorm`), but sometimes the 30 observations we get in our experiment will suggest that the mean is higher or lower. In this case, we'll get a "significant" result and incorrectly reject the null hypothesis of mean 0.

What's the proportion of "significant" results ($p < .05$) that you see?

First do this using a `for` loop.

```{r}
```

In [ ]:
significant_count = 0
for _ in range(30):
    dist = np.random.normal(size=30)
    ttest_result = stats.ttest_1samp(dist, 0)
    print(ttest_result)
    if ttest_result.pvalue < 0.05:
        significant_count += 1

print(significant_count)


Next, do this using the `replicate` function:

```{r}
```


In [ ]:
# Not available

How does this compare to the intended false-positive rate of $\alpha=0.05$?

> It's as expected to have false positive rate around 5%

Ok, that was a bit boring. Let's try something more interesting - let's implement a p-value sniffing simulation, in the style of Simons, Nelson, & Simonsohn (2011).

Consider this scenario: you have done an experiment, again with 30 participants (one observation each, just for simplicity). The question is whether the true mean is different from 0. You aren't going to check the p-value every trial, but let's say you run 30 - then if the p-value is within the range p < .25 and p > .05, you optionally run 30 more and add those data, then test again. But if the original p value is < .05, you call it a day, and if the original is > .25, you also stop.  

First, write a function that implements this sampling regime.

```{r}
double.sample <- function () {
}
```

In [ ]:
def double_sample(threshold=0.25):
    dist = np.random.normal(size=30)
    ttest_result = stats.ttest_1samp(dist, 0)
    if ttest_result.pvalue < 0.05:
        return True
    elif ttest_result.pvalue < threshold:
        dist_add = np.random.normal(size=30)
        dist = np.concatenate((dist, dist_add))
        ttest_result = stats.ttest_1samp(dist, 0)
        if ttest_result.pvalue < 0.05:
            return True
        else:
            return False


Now call this function 10k times and find out what happens. 

```{r}
```


In [ ]:
def test_false_positives(threshold=0.25):
    false_positives = 0
    trial = 10000
    for i in range(trial):
        if double_sample(threshold):
            false_positives += 1

    print(f"False positive rate: {false_positives / trial}")
test_false_positives()


Is there an inflation of false positives? How bad is it?


>The false positive rate is 7% instead of 5%

Now modify this code so that you can investigate this "double the sample" rule in a bit more depth. In the previous question, the researcher doubles the sample only when they think they got "close" to a significant result, i.e. when their not-significant p is less than 0.25. What if the researcher was more optimistic? See what happens in these 3 other scenarios:

* The researcher doubles the sample whenever their pvalue is not significant, but it's less than 0.5.
* The researcher doubles the sample whenever their pvalue is not significant, but it's less than 0.75.
* The research doubles their sample whenever they get ANY pvalue that is not significant.

How do these choices affect the false positive rate?

HINT: Try to do this by making the function `double.sample` take the upper p value as an argument, so that you can pass this through dplyr.

HINT 2: You may need more samples. Find out by looking at how the results change from run to run.

```{r}
```


In [ ]:
test_false_positives(0.5)
test_false_positives(0.75)
test_false_positives(1.0)


What do you conclude on the basis of this simulation? How bad is this kind of data-dependent policy?

> By increasing the sample size after looking at the statistics, the false positive rate increases.
> The false positive rate increases as the researchers increase the threshold when they double the sample size.
> The increase effect saturates quickly after when the researcher increase the threshold beyond 0.5.